Data

In [1]:
import torch.utils.data as data
from os import listdir
from os.path import join
from PIL import Image
from os.path import exists, join, basename
from os import remove
from six.moves import urllib
import tarfile
from torchvision.transforms import Compose, CenterCrop, ToTensor, Scale

In [2]:
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg"])
def load_img(filepath):
    img = Image.open(filepath).convert('YCbCr')
    y, _, _ = img.split()
    return y

In [3]:
class DatasetFromFolder(data.Dataset):
    def __init__(self, image_dir, input_transform=None, target_transform=None):
        super(DatasetFromFolder, self).__init__()
        self.image_filenames = [join(image_dir, x) for x in listdir(image_dir) if is_image_file(x)]

        self.input_transform = input_transform
        self.target_transform = target_transform
    def __getitem__(self, index):
        input_image = load_img(self.image_filenames[index])
        target = input_image.copy()
        if self.input_transform:
            input_image = self.input_transform(input_image)
        if self.target_transform:
            target = self.target_transform(target)
        return input_image, target
    def __len__(self):
        return len(self.image_filenames)

In [4]:
def download_bsd300(dest="./dataset"):
    output_image_dir = join(dest, "BSDS300/images")

    if not exists(output_image_dir):
        url = "http://www2.eecs.berkeley.edu/Research/Projects/CS/vision/bsds/BSDS300-images.tgz"
        print("downloading url ", url)

        data = urllib.request.urlopen(url)

        file_path = join(dest, basename(url))
        with open(file_path, 'wb') as f:
            f.write(data.read())

        print("Extracting data")
        with tarfile.open(file_path) as tar:
            for item in tar:
                tar.extract(item, dest)

        remove(file_path)

    return output_image_dir


def calculate_valid_crop_size(crop_size, upscale_factor):
    return crop_size - (crop_size % upscale_factor)


def input_transform(crop_size, upscale_factor):
    return Compose([
        CenterCrop(crop_size),
        Scale(crop_size // upscale_factor),
        ToTensor(),
    ])


def target_transform(crop_size):
    return Compose([
        CenterCrop(crop_size),
        ToTensor(),
    ])


def get_training_set(upscale_factor):
    root_dir = download_bsd300()
    train_dir = join(root_dir, "train")
    crop_size = calculate_valid_crop_size(256, upscale_factor)

    return DatasetFromFolder(train_dir,
                             input_transform=input_transform(crop_size, upscale_factor),
                             target_transform=target_transform(crop_size))


def get_test_set(upscale_factor):
    root_dir = download_bsd300()
    test_dir = join(root_dir, "test")
    crop_size = calculate_valid_crop_size(256, upscale_factor)

    return DatasetFromFolder(test_dir,
                             input_transform=input_transform(crop_size, upscale_factor),
                             target_transform=target_transform(crop_size))

model

In [5]:
import torch
import torch.nn as nn
class Net(torch.nn.Module):
    def __init__(self, num_channels, upscale_factor, d=64, s=12, m=4):
        super(Net, self).__init__()

        # Feature extraction
        self.first_part = nn.Sequential(nn.Conv2d(in_channels=num_channels, out_channels=d, kernel_size=5, stride=1, padding=2),
                                        nn.PReLU())

        self.layers = []
        # Shrinking
        self.layers.append(nn.Sequential(nn.Conv2d(in_channels=d, out_channels=s, kernel_size=1, stride=1, padding=0),
                                         nn.PReLU()))
        # Non-linear Mapping
        for _ in range(m):
            self.layers.append(nn.Conv2d(in_channels=s, out_channels=s, kernel_size=3, stride=1, padding=1))
        self.layers.append(nn.PReLU())
        # Expanding
        self.layers.append(nn.Sequential(nn.Conv2d(in_channels=s, out_channels=d, kernel_size=1, stride=1, padding=0),
                                         nn.PReLU()))

        self.mid_part = torch.nn.Sequential(*self.layers)

        # Deconvolution
        self.last_part = nn.ConvTranspose2d(in_channels=d, out_channels=num_channels, kernel_size=9, stride=upscale_factor, padding=3, output_padding=1)

    def forward(self, x):
        out = self.first_part(x)
        out = self.mid_part(out)
        out = self.last_part(out)
        return out

    def weight_init(self, mean=0.0, std=0.02):
        for m in self.modules():
            # utils.weights_init_normal(m, mean=mean, std=std)
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(mean, std)
                if m.bias is not None:
                    m.bias.data.zero_()
            if isinstance(m, nn.ConvTranspose2d):
                m.weight.data.normal_(0.0, 0.0001)
                if m.bias is not None:
                    m.bias.data.zero_()

train_and_test

In [6]:
import sys
import time
TOTAL_BAR_LENGTH = 80
LAST_T = time.time()
BEGIN_T = LAST_T
def progress_bar(current, total, msg=None):
    global LAST_T, BEGIN_T
    if current == 0:
        BEGIN_T = time.time()  # Reset for new bar.

    current_len = int(TOTAL_BAR_LENGTH * (current + 1) / total)
    rest_len = int(TOTAL_BAR_LENGTH - current_len) - 1

    sys.stdout.write(' %d/%d' % (current + 1, total))
    sys.stdout.write(' [')
    for i in range(current_len):
        sys.stdout.write('=')
    sys.stdout.write('>')
    for i in range(rest_len):
        sys.stdout.write('.')
    sys.stdout.write(']')

    current_time = time.time()
    step_time = current_time - LAST_T
    LAST_T = current_time
    total_time = current_time - BEGIN_T

    time_used = '  Step: %s' % format_time(step_time)
    time_used += ' | Tot: %s' % format_time(total_time)
    if msg:
        time_used += ' | ' + msg

    msg = time_used
    sys.stdout.write(msg)

    if current < total - 1:
        sys.stdout.write('\r')
    else:
        sys.stdout.write('\n')
    sys.stdout.flush()
# return the formatted time
def format_time(seconds):
    days = int(seconds / 3600/24)
    seconds = seconds - days*3600*24
    hours = int(seconds / 3600)
    seconds = seconds - hours*3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes*60
    seconds_final = int(seconds)
    seconds = seconds - seconds_final
    millis = int(seconds*1000)
    output = ''
    time_index = 1
    if days > 0:
        output += str(days) + 'D'
        time_index += 1
    if hours > 0 and time_index <= 2:
        output += str(hours) + 'h'
        time_index += 1
    if minutes > 0 and time_index <= 2:
        output += str(minutes) + 'm'
        time_index += 1
    if seconds_final > 0 and time_index <= 2:
        output += str(seconds_final) + 's'
        time_index += 1
    if millis > 0 and time_index <= 2:
        output += str(millis) + 'ms'
        time_index += 1
    if output == '':
        output = '0ms'
    return output

In [7]:
from __future__ import print_function
from math import log10

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
#from FSRCNN.model import Net
#from misc import progress_bar
from torch.optim import lr_scheduler

In [12]:
class FSRCNNTrainer(object):
    def __init__(self, training_loader, testing_loader):
        self.model = None
        self.lr = 0.1
        self.nEpochs = 200
        self.criterion = None
        self.optimizer = None
        self.scheduler = None
        #self.GPU_IN_USE = torch.cuda.is_available()
        self.seed = 111
        self.upscale_factor = 4
        self.training_loader = training_loader
        self.testing_loader = testing_loader

    def build_model(self):
        self.model = Net(num_channels=1, upscale_factor=self.upscale_factor)
        self.model.weight_init(mean=0.0, std=0.2)
        self.criterion = nn.MSELoss()
        torch.manual_seed(self.seed)

        #if self.GPU_IN_USE:
            #torch.cuda.manual_seed(self.seed)
            #self.model.cuda()
            #cudnn.benchmark = True
            #self.criterion.cuda()

        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.scheduler = lr_scheduler.MultiStepLR(self.optimizer, milestones=[50, 75, 100], gamma=0.5)  # lr decay

    def save(self):
        model_out_path = "FSRCNN_model_path.pth"
        torch.save(self.model, model_out_path)
        print("Checkpoint saved to {}".format(model_out_path))

    def train(self):
        """
        data: [torch.cuda.FloatTensor], 4 batches: [64, 64, 64, 8]
        """
        self.model.train()
        train_loss = 0
        for batch_num, (data, target) in enumerate(self.training_loader):
            #if self.GPU_IN_USE:
                #data, target = Variable(data).cuda(), Variable(target).cuda()
            data, target = Variable(data), Variable(target)
            self.optimizer.zero_grad()
            loss = self.criterion(self.model(data), target)
            train_loss += loss.data[0]
            loss.backward()
            self.optimizer.step()
            progress_bar(batch_num, len(self.training_loader), 'Loss: %.4f' % (train_loss / (batch_num + 1)))

        print("    Average Loss: {:.4f}".format(train_loss / len(self.training_loader)))

    def test(self):
        """
        data: [torch.cuda.FloatTensor], 10 batches: [10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
        """
        self.model.eval()
        avg_psnr = 0
        for batch_num, (data, target) in enumerate(self.testing_loader):
            #if self.GPU_IN_USE:
                #data, target = Variable(data).cuda(), Variable(target).cuda()
            data, target = Variable(data), Variable(target)
            prediction = self.model(data)
            mse = self.criterion(prediction, target)
            psnr = 10 * log10(1 / mse.data[0])
            avg_psnr += psnr
            progress_bar(batch_num, len(self.testing_loader), 'PSNR: %.4f' % (avg_psnr / (batch_num + 1)))

        print("    Average PSNR: {:.4f} dB".format(avg_psnr / len(self.testing_loader)))

    def validate(self):
        self.build_model()
        for epoch in range(1, self.nEpochs + 1):
            print("\n===> Epoch {} starts:".format(epoch))
            self.train()
            self.test()
            self.scheduler.step(epoch)
            if epoch == self.nEpochs:
                self.save()

begin

In [13]:
train_set = get_training_set(4)
test_set = get_test_set(4)
training_data_loader = data.DataLoader(dataset=train_set, batch_size=2, shuffle=True)
testing_data_loader = data.DataLoader(dataset=test_set, batch_size=2, shuffle=False)
model = FSRCNNTrainer(training_data_loader, testing_data_loader)
model.validate()


===> Epoch 1 starts:
 100/100 [================================================================================>]  Step: 107ms | Tot: 10s904ms | Loss: 40493820141920248.0000======================>...................................]  Step: 99ms | Tot: 6s152ms | Loss: 2652524574925373.0000===============>...............................]  Step: 99ms | Tot: 6s576ms | Loss: 3027821085219760.5000======================================================>.........................]  Step: 101ms | Tot: 7s430ms | Loss: 56275950814099088.0000 [==========================================================>.....................]  Step: 98ms | Tot: 7s956ms | Loss: 54997696445147440.0000 [========================================================================>.......]  Step: 99ms | Tot: 9s890ms | Loss: 44458609486534624.0000
    Average Loss: 40493820141920248.0000
 50/50 [================================================================================>]  Step: 58ms | Tot: 2s675ms | PSNR: -131.7692======

 50/50 [================================================================================>]  Step: 50ms | Tot: 2s584ms | PSNR: -119.9412...........................]  Step: 57ms | Tot: 868ms | PSNR: -120.3601.....]  Step: 41ms | Tot: 909ms | PSNR: -120.3903...............................................]  Step: 41ms | Tot: 1s8ms | PSNR: -120.2254............................................]  Step: 47ms | Tot: 1s111ms | PSNR: -120.2044.....]  Step: 45ms | Tot: 1s404ms | PSNR: -119.9255 [=========================================================>......................]  Step: 55ms | Tot: 1s831ms | PSNR: -119.8818
    Average PSNR: -119.9412 dB

===> Epoch 8 starts:
 100/100[===============================================================================>]  Step: 99ms | Tot: 10s370ms | Loss: 25053333026547.07038......................................................................]  Step: 99ms | Tot: 649ms | Loss: 1037327260525.7142 [========>..................................................

 100/100 [================================================================================>]  Step: 107ms | Tot: 10s574ms | Loss: 63497977917.4400>...........................................................................]  Step: 99ms | Tot: 551ms | Loss: 49893768533.3333=====>............................................................]  Step: 99ms | Tot: 2s462ms | Loss: 53520533162.6667.........................................]  Step: 98ms | Tot: 2s771ms | Loss: 54813604901.9259========================>.......................................................]  Step: 97ms | Tot: 3s188ms | Loss: 55796639479.7419 [============================>...................................................]  Step: 98ms | Tot: 3s607ms | Loss: 56097358116.5714 [============================================>...................................]  Step: 97ms | Tot: 5s915ms | Loss: 54123024420.5714 [====================================================>...........................]  Step: 99ms | Tot: 6s857ms 

 100/100 [================================================================================>]  Step: 105ms | Tot: 13s107ms | Loss: 165687506432.0000.................................]  Step: 111ms | Tot: 113ms | Loss: 30516453376.0000  Step: 200ms | Tot: 1s594ms | Loss: 36840733774.7692........................................]  Step: 96ms | Tot: 2s140ms | Loss: 47600709176.8889.....................]  Step: 98ms | Tot: 3s698ms | Loss: 74633709056.0000....]  Step: 95ms | Tot: 3s794ms | Loss: 91421982720.0000.........................]  Step: 95ms | Tot: 3s890ms | Loss: 99414861703.5294================>...................................................]  Step: 102ms | Tot: 3s992ms | Loss: 100555154110.1714========>................................................]  Step: 194ms | Tot: 4s675ms | Loss: 197177656346.2564 [====================================================>...........................]  Step: 202ms | Tot: 8s296ms | Loss: 233973152500.1846=======================================>.

 50/50 [================================================================================>]  Step: 37ms | Tot: 2s204ms | PSNR: -116.0882.......................................................................]  Step: 67ms | Tot: 148ms | PSNR: -117.5492................]  Step: 40ms | Tot: 503ms | PSNR: -117.5018 [===========================================>....................................]  Step: 35ms | Tot: 1s227ms | PSNR: -116.1775===============>...................................]  Step: 41ms | Tot: 1s269ms | PSNR: -116.1316..]  Step: 46ms | Tot: 1s580ms | PSNR: -116.0712.........]  Step: 46ms | Tot: 1s727ms | PSNR: -116.0373  Step: 41ms | Tot: 1s768ms | PSNR: -116.0736===================================================================>............]  Step: 50ms | Tot: 1s871ms | PSNR: -116.1409=================================================================>...........]  Step: 54ms | Tot: 1s925ms | PSNR: -116.1563
    Average PSNR: -116.0882 dB

===> Epoch 24 starts:
 100/100[====

...

 50/50 [================================================================================>]  Step: 48ms | Tot: 2s628ms | PSNR: -103.8887====>.......................................................................]  Step: 69ms | Tot: 211ms | PSNR: -104.3114 [====================================================>...........................]  Step: 49ms | Tot: 1s779ms | PSNR: -104.0781
    Average PSNR: -103.8887 dB

===> Epoch 175 starts:
 100/100 [==============================================================================>]  Step: 98ms | Tot: 10s365ms | Loss: 42995055347.07078============================================>..................................]  Step: 98ms | Tot: 5s957ms | Loss: 41134870545.9649================================>...............................]  Step: 96ms | Tot: 6s266ms | Loss: 41111421969.0667 [=====================================================>..........................]  Step: 101ms | Tot: 6s993ms | Loss: 41080882756.7761==>.......................]  Ste

 100/100 [================================================================================>]  Step: 113ms | Tot: 10s499ms | Loss: 20009085440.0000..........................]  Step: 97ms | Tot: 3s632ms | Loss: 28443106216.2286................................................]  Step: 99ms | Tot: 3s949ms | Loss: 28072222746.9474================================>...............................................]  Step: 98ms | Tot: 4s260ms | Loss: 27626950755.9024==============================>............................................]  Step: 99ms | Tot: 4s568ms | Loss: 26880149690.1818 [======================================>.........................................]  Step: 100ms | Tot: 4s987ms | Loss: 25561193237.3333========================================>.......................................]  Step: 99ms | Tot: 5s301ms | Loss: 24955601277.4902=========================================>....................................]  Step: 98ms | Tot: 5s613ms | Loss: 24358232082.9630........]  St

 50/50 [================================================================================>]  Step: 55ms | Tot: 2s640ms | PSNR: -99.9814......................................]  Step: 58ms | Tot: 140ms | PSNR: -101.2534........]  Step: 43ms | Tot: 1s975ms | PSNR: -99.9602==================================================>............]  Step: 53ms | Tot: 2s219ms | PSNR: -100.0749======================================================================>......]  Step: 48ms | Tot: 2s420ms | PSNR: -100.0266
    Average PSNR: -99.9814 dB

===> Epoch 189 starts:
 100/100 [================================================================================>]  Step: 105ms | Tot: 10s396ms | Loss: 8626299985.9200.......................................................]  Step: 98ms | Tot: 512ms | Loss: 8735199914.6667 [=======>........................................................................]  Step: 97ms | Tot: 833ms | Loss: 8143176362.6667 [==========>.................................................

 50/50 [================================================================================>]  Step: 45ms | Tot: 2s483ms | PSNR: -97.2474...............................................]  Step: 42ms | Tot: 326ms | PSNR: -98.0883==============>.........................................................]  Step: 57ms | Tot: 683ms | PSNR: -98.1144==================================================>.......................]  Step: 53ms | Tot: 1s752ms | PSNR: -97.2066=============================>...........]  Step: 46ms | Tot: 2s157ms | PSNR: -97.3867
    Average PSNR: -97.2474 dB

===> Epoch 195 starts:
 100/100 [================================================================================>]  Step: 111ms | Tot: 10s509ms | Loss: 5983260738.5600==>.......................................................................]  Step: 100ms | Tot: 953ms | Loss: 6529537510.4000 [=============================================================>..................]  Step: 100ms | Tot: 8s61ms | Loss: 6079003362.0

/home/liar/anaconda3/envs/python3/lib/python3.5/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
